In [2]:
# But : Trouver des codes NAFS à partir de mots-clés (inclus dans les codes ROMES / code NAF avec noms/descriptions
#
# Avis: ce n'est pas parfait encore en terme de résultat mais le but est de s'y "rapprocher"
#


# Avec sentences transformers

import pandas as pd
from sentence_transformers import SentenceTransformer, util
from chromadb import PersistentClient
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import pandas as pd

# Charger le modèle d'embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Chroma en mode persistant
client = PersistentClient(path="./chroma_db")

# Charger ou créer la collection
collection = client.get_or_create_collection("naf")

desc_embeddings = None

# Charger le fichier fusionné
print("Chargement du fichier NAF et code ROME...")
# df = pd.read_csv("fusion_naf_rome_001.csv")
df = pd.read_csv("fusion_naf_rome_001_allMiniLM_L6_v2.csv")
# df = pd.read_csv("fusion_naf_rome_001_thenlper_gte-large.csv")

# Vérifier si les embeddings sont déjà stockés
if collection.count() > 0:
    print("✔ Embeddings déjà existants, rien à faire")
    results = collection.get(include=["embeddings"])
    # Liste des embeddings (list of list of floats)
    desc_embeddings = results["embeddings"]
else:
    print("⚠ Pas d'embeddings, génération en cours...")

    # Nettoyer la colonne desc
    df['desc'] = df['desc'].astype(str)

    print("Calcul des embeddings avec le model all-MiniLM-L6-v2...")
    # vectors = model.encode(df['desc'].tolist(), convert_to_tensor=True)
    vectors = model.encode(df['desc'].tolist(), convert_to_numpy=True, normalize_embeddings=True)

    print("Début stockage des Embeddings dans Chroma...")
    BATCH_SIZE = 2000

    ids = [str(i) for i in df.index]
    docs = df["desc"].tolist()

    for start in range(0, len(vectors), BATCH_SIZE):
        end = start + BATCH_SIZE
        print(f"Ajout du batch {start} → {end}")

        collection.add(
            ids=ids[start:end],
            embeddings=vectors[start:end],
            documents=docs[start:end]
        )
    print("Fin du stockage des Embeddings dans Chroma.")

    # Créer les embeddings pour les descriptions du CSV
    print("Extraction des embeddings en mémoire pour le calcul...")
    desc_embeddings = vectors

"""
OUTPUT:

⚠ Pas d'embeddings, génération en cours...
Chargement du fichier NAF et code ROME...
Calcul des embeddings avec le model all-MiniLM-L6-v2...
Début stockage des Embeddings dans Chroma...
Ajout du batch 0 → 5000
Ajout du batch 5000 → 10000
Ajout du batch 10000 → 15000
Fin du stockage des Embeddings dans Chroma.
tensor([[-0.0980,  0.0399,  0.0037,  ...,  0.0069,  0.0999, -0.0349],
        [-0.1088, -0.0010, -0.0013,  ...,  0.1023,  0.0740, -0.0113],
        [-0.1048,  0.0603, -0.0711,  ...,  0.0598,  0.0491,  0.0329],
        ...,
        [-0.0575,  0.0009, -0.0259,  ...,  0.0897,  0.0452, -0.0117],
        [-0.0615, -0.0122, -0.0230,  ...,  0.0841,  0.0256, -0.0115],
        [-0.0600, -0.0323, -0.0460,  ...,  0.0289,  0.0006, -0.0440]])

"""

desc_embeddings

Chargement du fichier NAF et code ROME...
⚠ Pas d'embeddings, génération en cours...
Calcul des embeddings avec le model all-MiniLM-L6-v2...
Début stockage des Embeddings dans Chroma...
Ajout du batch 0 → 2000
Ajout du batch 2000 → 4000
Ajout du batch 4000 → 6000
Ajout du batch 6000 → 8000
Ajout du batch 8000 → 10000
Ajout du batch 10000 → 12000
Fin du stockage des Embeddings dans Chroma.
Extraction des embeddings en mémoire pour le calcul...


array([[-0.09797601,  0.0399228 ,  0.00369266, ...,  0.00687646,
         0.0998799 , -0.03494112],
       [-0.10879397, -0.00104912, -0.00128806, ...,  0.10225021,
         0.07395095, -0.01129509],
       [-0.10484292,  0.06034279, -0.07107382, ...,  0.05976397,
         0.04913333,  0.0329356 ],
       ...,
       [-0.057535  ,  0.00089183, -0.02591405, ...,  0.08968923,
         0.04520449, -0.01169417],
       [-0.06149168, -0.01219322, -0.02297172, ...,  0.08405706,
         0.02563769, -0.01146613],
       [-0.0599971 , -0.03229763, -0.0460492 , ...,  0.02885329,
         0.00060947, -0.04403988]], shape=(11858, 384), dtype=float32)

In [3]:
# Créer un embedding pour la recherche
query = "professeur yoga"
query_embedding = model.encode(query, convert_to_tensor=True)
# query_embedding = query_embedding.double()

# Calculer la similarité cosinus
cos_scores = util.cos_sim(query_embedding, desc_embeddings)[0]

# Récupérer les indices triés par similarité décroissante
top_indices = cos_scores.argsort(descending=True)

In [4]:
import pandas as pd

# Filtrer les résultats pertinents (par ex. score > 0.3)
results = []
threshold = 0.45
# for idx in top_indices:
#     score = cos_scores[idx].item()
#     if score >= threshold:
#         results.append((score, df.iloc[idx]))
#     else:
#         break

for idx in top_indices:
    score = cos_scores[idx].item()
    if score >= threshold:
        results.append((score, df.iloc[int(idx)]))  # <-- conversion en int
    else:
        break



# Afficher les résultats sous forme de tableau
search_results = pd.DataFrame([r[1] for r in results])
print(search_results)
search_results

     code_naf code_rome                                        name  \
3602   96.04Z     G1209  Animateur / Animatrice de loisirs sportifs   

                                                   desc  
3602  Professeur / Professeure de yoga (code OGR:181...  


,code_naf,code_rome,name,desc
3602,96.04Z,G1209,Animateur / Animatrice de loisirs sportifs,Professeur / Professeure de yoga (code OGR:181...


In [5]:
# Avec recherche simple

import pandas as pd

print("Chargement du fichier NAF et code ROME...")
# df = pd.read_csv("fusion_naf_rome_001.csv")
df = pd.read_csv("fusion_naf_rome_001_allMiniLM_L6_v2.csv")
# df = pd.read_csv("fusion_naf_rome_001_thenlper_gte-large.csv")

# Fonction de recherche par mots-clés
def search_keywords(df, keywords):
    # Transformer la requête en liste de mots
    words = keywords.lower().split()

    # Filtrer les lignes contenant tous les mots-clés dans 'desc' ou 'name'
    mask = df.apply(
        lambda row: all(word in str(row['desc']).lower() or word in str(row['name']).lower() for word in words),
        axis=1
    )
    results = df[mask]

    # Si plusieurs résultats, ne garder que ceux sans code_rome
    if len(results) > 1:
        results_no_rome = results[results['code_rome'].isna() | (results['code_rome'] == "")]
        if not results_no_rome.empty:
            results = results_no_rome

    return results

# Exemple de recherche
keywords = "professeur yoga"
results = search_keywords(df, keywords)
# Afficher le résultat
print(results)
# Sauvegarder éventuellement dans un CSV
# results.to_csv("search_results.csv", index=False)


Chargement du fichier NAF et code ROME...
      code_naf code_rome                                               name  \
11783   85.51Z       NaN  Enseignement de disciplines sportives et d'act...   

                                                    desc  
11783  85.51Z Enseignement de disciplines sportives e...  


In [6]:
keywords = "bar restaurant"
results = search_keywords(df, keywords)
results

,code_naf,code_rome,name,desc
11644,56.10A,NaN,Restauration traditionnelle,56.10A Restauration traditionnelle Cette sous-...


In [22]:
keywords = "débit boisson"
results = search_keywords(df, keywords)
results

,code_naf,code_rome,name,desc
11572,47.25Z,NaN,Commerce de détail de boissons en magasin spéc...,47.25Z Commerce de détail de boissons en magas...
11649,56.29B,NaN,Autres services de restauration n.c.a.,56.29B Autres services de restauration n.c.a. ...
11650,56.30Z,NaN,Débits de boissons,56.30.10 Services de débits de boissons CC : -...


In [7]:
# Exemple de recherche
keywords = "professeur yoga"
results = search_keywords(df, keywords)
results

,code_naf,code_rome,name,desc
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...


In [8]:
keywords = "fast food"
results = search_keywords(df, keywords)
results

,code_naf,code_rome,name,desc
3793,56.10C,G1409,Manager en restauration rapide,Gérant / Gérante de fast-food (code OGR:488969...


In [9]:
keywords = "traiteur"
results = search_keywords(df, keywords)
results

,code_naf,code_rome,name,desc
11647,56.21Z,NaN,Services des traiteurs,56.21Z Services des traiteurs Cette sous-class...


In [10]:
keywords = "bar"
results = search_keywords(df, keywords)
results

,code_naf,code_rome,name,desc
11145,01.43Z,NaN,Élevage de chevaux et d'autres équidés,01.43Z Élevage de chevaux et d'autres équidés ...
11155,01.64Z,NaN,Traitement des semences,01.64Z Traitement des semences Cette sous-clas...
11174,08.91Z,NaN,Extraction des minéraux chimiques et d'engrais...,08.91Z Extraction des minéraux chimiques et d'...
11250,16.23Z,NaN,Fabrication de charpentes et d'autres menuiseries,16.23Z Fabrication de charpentes et d'autres m...
11299,23.19Z,NaN,Fabrication et façonnage d'autres articles en ...,23.19Z Fabrication et façonnage d'autres artic...
11319,24.10Z,NaN,Sidérurgie,24.10Z Sidérurgie Cette sous-classe comprend :...
11321,24.31Z,NaN,Étirage à froid de barres,24.31.99 Opérations sous-traitées intervenant ...
11335,25.11Z,NaN,Fabrication de structures métalliques et de pa...,25.11Z Fabrication de structures métalliques e...
11352,25.93Z,NaN,"Fabrication d'articles en fils métalliques, de...","25.93.11 Torons, câbles, tresses, élingues et ..."
11358,26.20Z,NaN,Fabrication d'ordinateurs et d'équipements pér...,26.20Z Fabrication d'ordinateurs et d'équipeme...


In [11]:
keywords = "boulangerie"
results = search_keywords(df, keywords)
results

,code_naf,code_rome,name,desc
11197,10.61A,NaN,Meunerie,10.61A Meunerie Cette sous-classe comprend : -...
11201,10.71B,NaN,Cuisson de produits de boulangerie,10.71B Cuisson de produits de boulangerie Cett...
11202,10.71C,NaN,Boulangerie et boulangerie-pâtisserie,10.71C Boulangerie et boulangerie-pâtisserie C...
11394,28.93Z,NaN,Fabrication de machines pour l'industrie agro-...,28.93.13 Machines et appareils pour la minoter...
11524,46.36Z,NaN,Commerce de gros (commerce interentreprises) d...,46.36Z Commerce de gros (commerce interentrepr...
11527,46.38B,NaN,Commerce de gros (commerce interentreprises) a...,46.38B Commerce de gros (commerce interentrepr...


In [12]:
import re

def search_keywords_strict(df, keywords):
    words = keywords.lower().split()
    pattern = r'\b' + r'\b.*\b'.join(re.escape(w) for w in words) + r'\b'

    mask = df.apply(
        lambda row: bool(re.search(pattern, str(row['desc']).lower()))
                    or bool(re.search(pattern, str(row['name']).lower())),
        axis=1
    )
    results = df[mask]

    # Filtre code_rome si plusieurs résultats
    if len(results) > 1:
        results_no_rome = results[results['code_rome'].isna() | (results['code_rome'] == "")]
        if not results_no_rome.empty:
            results = results_no_rome
    return results

def search_keywords_ranked(df, keywords):
    words = keywords.lower().split()

    def score(row):
        text = (str(row['desc']) + " " + str(row['name'])).lower()
        return sum(word in text for word in words)

    df['score'] = df.apply(score, axis=1)
    results = df[df['score'] > 0].sort_values(by='score', ascending=False)

    # Filtre code_rome si plusieurs résultats
    if len(results) > 1:
        results_no_rome = results[results['code_rome'].isna() | (results['code_rome'] == "")]
        if not results_no_rome.empty:
            results = results_no_rome
    return results.drop(columns=['score'])


In [13]:
keywords = "boulangerie"
results = search_keywords(df, keywords)
results

,code_naf,code_rome,name,desc
11197,10.61A,NaN,Meunerie,10.61A Meunerie Cette sous-classe comprend : -...
11201,10.71B,NaN,Cuisson de produits de boulangerie,10.71B Cuisson de produits de boulangerie Cett...
11202,10.71C,NaN,Boulangerie et boulangerie-pâtisserie,10.71C Boulangerie et boulangerie-pâtisserie C...
11394,28.93Z,NaN,Fabrication de machines pour l'industrie agro-...,28.93.13 Machines et appareils pour la minoter...
11524,46.36Z,NaN,Commerce de gros (commerce interentreprises) d...,46.36Z Commerce de gros (commerce interentrepr...
11527,46.38B,NaN,Commerce de gros (commerce interentreprises) a...,46.38B Commerce de gros (commerce interentrepr...


In [14]:
keywords = "boulangerie"
results = search_keywords_strict(df, keywords)
results

,code_naf,code_rome,name,desc
11197,10.61A,NaN,Meunerie,10.61A Meunerie Cette sous-classe comprend : -...
11201,10.71B,NaN,Cuisson de produits de boulangerie,10.71B Cuisson de produits de boulangerie Cett...
11202,10.71C,NaN,Boulangerie et boulangerie-pâtisserie,10.71C Boulangerie et boulangerie-pâtisserie C...
11394,28.93Z,NaN,Fabrication de machines pour l'industrie agro-...,28.93.13 Machines et appareils pour la minoter...
11524,46.36Z,NaN,Commerce de gros (commerce interentreprises) d...,46.36Z Commerce de gros (commerce interentrepr...
11527,46.38B,NaN,Commerce de gros (commerce interentreprises) a...,46.38B Commerce de gros (commerce interentrepr...


In [15]:
keywords = "boulangerie"
results = search_keywords_ranked(df, keywords)
results

,code_naf,code_rome,name,desc
11197,10.61A,NaN,Meunerie,10.61A Meunerie Cette sous-classe comprend : -...
11201,10.71B,NaN,Cuisson de produits de boulangerie,10.71B Cuisson de produits de boulangerie Cett...
11202,10.71C,NaN,Boulangerie et boulangerie-pâtisserie,10.71C Boulangerie et boulangerie-pâtisserie C...
11394,28.93Z,NaN,Fabrication de machines pour l'industrie agro-...,28.93.13 Machines et appareils pour la minoter...
11524,46.36Z,NaN,Commerce de gros (commerce interentreprises) d...,46.36Z Commerce de gros (commerce interentrepr...
11527,46.38B,NaN,Commerce de gros (commerce interentreprises) a...,46.38B Commerce de gros (commerce interentrepr...


In [16]:
keywords = "professeur de yoga"
results = search_keywords_strict(df, keywords)
results

,code_naf,code_rome,name,desc,score
3602,96.04Z,G1209,Animateur / Animatrice de loisirs sportifs,Professeur / Professeure de yoga (code OGR:181...,0


In [17]:
keywords = "yoga"
results = search_keywords_strict(df, keywords)
results

,code_naf,code_rome,name,desc,score
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...,0


In [18]:
import pandas as pd
import re

def search_keywords_flexible(df, keywords, filter_code_rome_threshold=3):
    """
    Recherche par mots-clés (mots complets) avec logique flexible sur code_rome.

    - keywords : str, mots-clés de recherche
    - filter_code_rome_threshold : nombre de résultats avant d'appliquer le filtre code_rome
    """
    words = keywords.lower().split()
    pattern = r'\b' + r'\b.*\b'.join(re.escape(w) for w in words) + r'\b'

    # Filtrer les lignes contenant tous les mots-clés
    mask = df.apply(
        lambda row: bool(re.search(pattern, str(row['desc']).lower()))
                    or bool(re.search(pattern, str(row['name']).lower())),
        axis=1
    )
    results = df[mask].copy()

    # Ajouter un score pour classer par pertinence (nombre de mots trouvés)
    results['score'] = results.apply(
        lambda row: sum(word in (str(row['desc']) + " " + str(row['name'])).lower() for word in words),
        axis=1
    )

    results = results.sort_values(by='score', ascending=False)

    # Filtre code_rome seulement si beaucoup de résultats
    if len(results) >= filter_code_rome_threshold:
        results_no_rome = results[results['code_rome'].isna() | (results['code_rome'] == "")]
        if not results_no_rome.empty:
            results = results_no_rome

    return results.drop(columns=['score'])


In [19]:
keywords = "yoga"
results = search_keywords_flexible(df, keywords)
results

,code_naf,code_rome,name,desc
3602,96.04Z,G1209,Animateur / Animatrice de loisirs sportifs,Professeur / Professeure de yoga (code OGR:181...
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...


In [20]:
keywords = "professeur de yoga"
results = search_keywords_flexible(df, keywords)
results

,code_naf,code_rome,name,desc
3602,96.04Z,G1209,Animateur / Animatrice de loisirs sportifs,Professeur / Professeure de yoga (code OGR:181...


In [21]:
keywords = "professeur yoga"
results = search_keywords_flexible(df, keywords)
results

,code_naf,code_rome,name,desc
3602,96.04Z,G1209,Animateur / Animatrice de loisirs sportifs,Professeur / Professeure de yoga (code OGR:181...


In [22]:
import pandas as pd
import re

def search_keywords_smart(df, keywords, filter_code_rome_threshold=3):
    words = keywords.lower().split()

    def relevance(row):
        text = (str(row['desc']) + " " + str(row['name'])).lower()
        # compter le nombre de mots clés présents
        return sum(word in text for word in words)

    # Calculer la pertinence
    df['score'] = df.apply(relevance, axis=1)

    # Garde toutes les lignes avec score > 0
    results = df[df['score'] > 0].copy()

    # Trier par pertinence (plus de mots-clés trouvés en premier)
    results = results.sort_values(by='score', ascending=False)

    # Filtre code_rome seulement si beaucoup de résultats
    if len(results) >= filter_code_rome_threshold:
        results_no_rome = results[results['code_rome'].isna() | (results['code_rome'] == "")]
        if not results_no_rome.empty:
            results = results_no_rome

    return results.drop(columns=['score'])


In [23]:
keywords = "professeur yoga"
results = search_keywords_smart(df, keywords)
results

,code_naf,code_rome,name,desc
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...
11746,78.30Z,NaN,Autre mise à disposition de ressources humaines,78.30.19 Autres services de mise à disposition...
11779,85.31Z,NaN,Enseignement secondaire général,85.31Z Enseignement secondaire général Cette s...
11784,85.52Z,NaN,Enseignement culturel,85.52Z Enseignement culturel Cette sous-classe...
11787,85.59B,NaN,Autres enseignements,85.59B Autres enseignements Cette sous-classe ...
11828,93.12Z,NaN,Activités de clubs de sports,93.12.10 Services de clubs de sports CC : - se...
11829,93.13Z,NaN,Activités des centres de culture physique,93.13Z Activités des centres de culture physiq...
11830,93.19Z,NaN,Autres activités liées au sport,93.19Z Autres activités liées au sport Cette s...


In [24]:
keywords = "yoga"
results = search_keywords_smart(df, keywords)
results

,code_naf,code_rome,name,desc
3602,96.04Z,G1209,Animateur / Animatrice de loisirs sportifs,Professeur / Professeure de yoga (code OGR:181...
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...


In [25]:
import pandas as pd
import re

def search_keywords_precise(df, keywords, filter_code_rome_threshold=3):
    words = keywords.lower().split()

    def relevance(row):
        text = (str(row['desc']) + " " + str(row['name'])).lower()
        # compter combien de mots clés exacts sont présents
        return sum(bool(re.search(r'\b{}\b'.format(re.escape(word)), text)) for word in words)

    df['score'] = df.apply(relevance, axis=1)
    # garder seulement les lignes qui ont au moins 1 mot clé exact
    results = df[df['score'] > 0].copy()
    # trier par score (plus de mots-clés trouvés en premier)
    results = results.sort_values(by='score', ascending=False)

    # Filtrer code_rome uniquement si beaucoup de résultats
    if len(results) >= filter_code_rome_threshold:
        results_no_rome = results[results['code_rome'].isna() | (results['code_rome'] == "")]
        if not results_no_rome.empty:
            results = results_no_rome

    return results.drop(columns=['score'])


In [26]:
keywords = "yoga"
results = search_keywords_precise(df, keywords)
results

,code_naf,code_rome,name,desc
3602,96.04Z,G1209,Animateur / Animatrice de loisirs sportifs,Professeur / Professeure de yoga (code OGR:181...
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...


In [27]:
keywords = "professeur yoga"
results = search_keywords_precise(df, keywords)
results

,code_naf,code_rome,name,desc
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...


In [28]:
keywords = "professeur de yoga"
results = search_keywords_precise(df, keywords)
results

,code_naf,code_rome,name,desc
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...
11841,95.21Z,NaN,Réparation de produits électroniques grand public,95.21.10 Services de réparation de produits él...
11857,99.00Z,NaN,Activités des organisations et organismes extr...,99.00Z Activités des organisations et organism...
11126,01.11Z,NaN,"Culture de céréales (à l'exception du riz), de...",01.11Z Culture de céréales (à l'exception du r...
11128,01.13Z,NaN,"Culture de légumes, de melons, de racines et d...",01.13.19 Autres légumes à feuilles ou à tiges ...
...,...,...,...,...
11853,96.09Z,NaN,Autres services personnels n.c.a.,96.09Z Autres services personnels n.c.a. Cette...
11854,97.00Z,NaN,Activités des ménages en tant qu'employeurs de...,97.00Z Activités des ménages en tant qu'employ...
11855,98.10Z,NaN,Activités indifférenciées des ménages en tant ...,98.10Z Activités indifférenciées des ménages e...
11856,98.20Z,NaN,Activités indifférenciées des ménages en tant ...,98.20Z Activités indifférenciées des ménages e...


In [29]:
import pandas as pd
import re

# Liste basique de stopwords
stopwords = {"de", "du", "la", "le", "et", "à", "des", "les", "en", "pour"}

def search_keywords_clean(df, keywords, filter_code_rome_threshold=3):
    # Séparer et filtrer les mots vides
    words = [w.lower() for w in keywords.split() if w.lower() not in stopwords]

    def relevance(row):
        text = (str(row['desc']) + " " + str(row['name'])).lower()
        # compter combien de mots significatifs sont présents
        return sum(bool(re.search(r'\b{}\b'.format(re.escape(word)), text)) for word in words)

    df['score'] = df.apply(relevance, axis=1)
    # garder seulement les lignes qui ont au moins 1 mot clé significatif
    results = df[df['score'] > 0].copy()
    # trier par pertinence
    results = results.sort_values(by='score', ascending=False)

    # Filtrer code_rome uniquement si beaucoup de résultats
    if len(results) >= filter_code_rome_threshold:
        results_no_rome = results[results['code_rome'].isna() | (results['code_rome'] == "")]
        if not results_no_rome.empty:
            results = results_no_rome

    return results.drop(columns=['score'])


In [30]:
keywords = "professeur de yoga"
results = search_keywords_clean(df, keywords)
results

,code_naf,code_rome,name,desc
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...


In [31]:
keywords = "professeur  yoga"
results = search_keywords_clean(df, keywords)
results

,code_naf,code_rome,name,desc
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...


In [32]:
keywords = "yoga"
results = search_keywords_clean(df, keywords)
results

,code_naf,code_rome,name,desc
3602,96.04Z,G1209,Animateur / Animatrice de loisirs sportifs,Professeur / Professeure de yoga (code OGR:181...
11783,85.51Z,NaN,Enseignement de disciplines sportives et d'act...,85.51Z Enseignement de disciplines sportives e...


In [33]:
keywords = "restaurant"
results = search_keywords_clean(df, keywords)
results

,code_naf,code_rome,name,desc
11412,31.01Z,NaN,Fabrication de meubles de bureau et de magasin,31.01.11 Mobilier métallique de bureau CC : - ...
11753,81.10Z,NaN,Activités combinées de soutien lié aux bâtiments,81.10Z Activités combinées de soutien lié aux ...
